In [1]:
using CSV
using DataFrames
using JuMP
using Gurobi
using Statistics

# ---------------------------------------------
# 1. Load baseline EV registrations (non-scenario dataset)
#    cleaned_registration_ny_with_coords.csv
# ---------------------------------------------
reg_raw = CSV.read("../data/cleaned_reg_1128.csv", DataFrame)

# Make sure ZIP is an Int (often it shows up as Float64 or String)
if eltype(reg_raw.ZIP) <: AbstractFloat
    reg_raw.ZIP = round.(Int, reg_raw.ZIP)
elseif eltype(reg_raw.ZIP) <: AbstractString
    reg_raw.ZIP = parse.(Int, strip.(reg_raw.ZIP))
end

# If there are multiple rows per ZIP, aggregate vehicle_count
# (if it's already one row per ZIP, this just passes through)
reg_zip = combine(
    groupby(reg_raw, :ZIP),
    :N => sum => :vehicle_count,
)

println("Number of ZIPs in registration file: ", nrow(reg_zip))

# ---------------------------------------------
# 2. Stations (ID, Status Code, total_evse)
# ---------------------------------------------
stations = CSV.read("../data/cleaned_station.csv", DataFrame)

stations.total_evse =
    coalesce.(stations[!, "EV Level1 EVSE Num"], 0.0) .+
    coalesce.(stations[!, "EV Level2 EVSE Num"], 0.0) .+
    coalesce.(stations[!, "EV DC Fast Count"], 0.0)

stations_small = select(stations, :ID, :"Status Code", :total_evse)

# ---------------------------------------------
# 3. Distance matrix dij (ZIP × station)
#    Structure: demand_zip | 35272 | 35562 | ...
# ---------------------------------------------
dist_raw = CSV.read("../data/dij.csv", DataFrame)

# Map ZIP → row index in dist_raw
zip_to_row = Dict(dist_raw.demand_zip[r] => r for r in 1:nrow(dist_raw))

# All station columns in the distance matrix
station_cols = names(dist_raw)[2:end]

# Helper: column name -> Int station ID (works for Symbols/Strings)
station_id_from_colname(name) = parse(Int, String(name))

# Station IDs as they appear in dij.csv
station_ids_from_dist = [station_id_from_colname(c) for c in station_cols]

# Restrict to stations that appear in BOTH station table and distance matrix
ids_from_stations = Set(stations_small.ID)
candidate_station_ids = [sid for sid in station_ids_from_dist if sid in ids_from_stations]

println("Number of candidate stations (intersection): ",
        length(candidate_station_ids))

# Create J index set based on candidate stations
station_ids = candidate_station_ids
J = collect(1:length(station_ids))
nJ = length(J)

# Map station_id -> column index in dist_raw
col_idx_for_station = Dict{Int,Int}()
for (k, col_name) in enumerate(station_cols)
    sid = station_id_from_colname(col_name)
    if sid in station_ids
        col_idx_for_station[sid] = k + 1   # +1 because col 1 is :demand_zip
    end
end

# ---------------------------------------------
# 4. Align ZIPs between registration file and dij,
#    and build baseline demand vector D and distance matrix d
# ---------------------------------------------

# Keep only ZIPs that appear in BOTH reg_zip and dist_raw
valid_zips = sort(
    collect(
        intersect(Set(reg_zip.ZIP), Set(dist_raw.demand_zip))
    )
)

println("ZIPs in both registration & dij: ", length(valid_zips))

# Internal demand-node index set
zip_codes = valid_zips              # external labels for demand nodes
I = collect(1:length(zip_codes))    # internal indices 1..|I|
nI = length(I)

# Build a dictionary ZIP -> vehicle_count
veh_dict = Dict(row.ZIP => row.vehicle_count for row in eachrow(reg_zip))

# Parameter D[i]: expected daily charging demand at ZIP i
sessions_per_EV_per_day = 0.25
D = [sessions_per_EV_per_day * veh_dict[z] for z in zip_codes]

println("Total baseline EV sessions/day (sum D): ", sum(D))

# Build dense distance matrix d[i,j] (in km) aligned with I and J
d = Matrix{Float64}(undef, nI, nJ)

for (i_idx, z) in enumerate(zip_codes)
    if !haskey(zip_to_row, z)
        error("ZIP $z from registration file not found in dij.csv; ",
              "either fix dij.csv or drop this ZIP.")
    end
    row = zip_to_row[z]
    for (j_idx, sid) in enumerate(station_ids)
        col = col_idx_for_station[sid]
        d[i_idx, j_idx] = dist_raw[row, col]
    end
end

println("Built distance matrix d of size ", size(d))

Number of ZIPs in registration file: 1064
Number of candidate stations (intersection): 5594
ZIPs in both registration & dij: 1064
Total baseline EV sessions/day (sum D): 320332.0
Built distance matrix d of size (1064, 5594)


In [2]:
# ==========================================================
# 2. Global parameters from your proposal
# ==========================================================

# Capacity per charger (sessions/day)
μ_per_charger = 3.65
μ = fill(μ_per_charger, nJ)          # µ_j is homogeneous

# Fixed & variable cost parameters
F = fill(25_000.0, nJ)               # F_j = $25,000
v = fill(15_000.0, nJ)               # v_j = $15,000 per charger

# Detour + environmental cost coefficients
β = 0.7                              # $/km
α = 0.013                            # $/km
cost_per_km = β + α                  # = 0.713 $/km

# Budget and big-M
B = 1_000_000_000.0                  # $1B
M = 50.0                             # max chargers per site (tunable upper bound)

max_chargers_per_station = 10

10

# Baseline 2022

In [3]:
using CSV
using DataFrames
using JuMP
using Gurobi

function solve_baseline_model(R_max_km::Float64;
                              write_outputs::Bool = false,
                              tag::String = "")

    # Penalty per unmet charging session
    λ_unmet = 15000.0   # tune as you like

    # ---------- 3.1 Feasible (i,j) pairs within R_max ----------
    edges = Tuple{Int,Int}[]
    for i in I, j in J
        if d[i,j] <= R_max_km
            push!(edges, (i,j))
        end
    end

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    println("R_max = $(R_max_km) km → feasible (i,j) pairs: ", length(edges))

    # ---------- 3.2 Build JuMP model ----------
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    set_optimizer_attribute(model, "TimeLimit", 600.0)
    set_optimizer_attribute(model, "MIPGap", 0.01)

    @variable(model, y[j in J], Bin)         # open station j
    @variable(model, z[j in J] >= 0, Int)    # chargers at j
    @variable(model, x[edges] >= 0)          # fraction of demand i served by j
    @variable(model, 0 .<= u[i in I] .<= 1)  # unmet demand fraction at i

    # ---------- 3.3 Constraints ----------
    # Demand balance: served + unmet = 1
    @constraint(model, demand_balance[i in I],
        sum(x[(i,j)] for j in N_i[i]) + u[i] == 1
    )

    # Capacity
    @constraint(model, capacity[j in J],
        sum(D[i] * x[(i,j)] for i in N_j[j]) <= μ[j] * z[j]
    )

    # Assignment link
    @constraint(model, assignment_link[(i,j) in edges],
        x[(i,j)] <= y[j]
    )

    # Charger–opening link + hard cap of 10 chargers per station
    @constraint(model, charger_link[j in J],
        z[j] <= M * y[j]
    )
    @constraint(model, charger_cap[j in J],
        z[j] <= 10
    )

    # Budget
    @constraint(model, budget,
        sum(F[j] * y[j] + v[j] * z[j] for j in J) <= B
    )

    # ---------- 3.4 Objective with unmet demand ----------
    @objective(model, Min,
        # infra cost
        sum(F[j] * y[j] + v[j] * z[j] for j in J) +
        # detour + env cost
        sum(cost_per_km * d[i,j] * D[i] * x[(i,j)] for (i,j) in edges) +
        # unmet demand penalty
        λ_unmet * sum(D[i] * u[i] for i in I)
    )

    # ---------- 3.5 Solve ----------
    optimize!(model)

    term_status = termination_status(model)
    println("Termination status: ", term_status)
    obj_val = objective_value(model)
    println("Objective value:    ", obj_val)

    # ---------- 3.6 Extract and summarize ----------
    open_sites_idx = [j for j in J if value(y[j]) > 0.5]
    num_open = length(open_sites_idx)
    total_chargers = sum(value(z[j]) for j in J)

    total_fixed = sum(F[j] * value(y[j]) for j in J)
    total_var   = sum(v[j] * value(z[j]) for j in J)
    total_detour_cost =
        sum(cost_per_km * d[i,j] * D[i] * value(x[(i,j)]) for (i,j) in edges)
    total_unmet_penalty =
        λ_unmet * sum(D[i] * value(u[i]) for i in I)

    total_detour_distance =
        sum(d[i,j] * D[i] * value(x[(i,j)]) for (i,j) in edges)

    total_served = sum(D[i] * (1.0 - value(u[i])) for i in I)
    total_unmet  = sum(D[i] * value(u[i]) for i in I)
    total_demand = total_served + total_unmet
    served_fraction = total_served / max(total_demand, 1e-9)

    avg_detour = total_detour_distance / max(total_served, 1e-9)

    println("---- Baseline summary for R_max = $(R_max_km) km ----")
    println("Open stations:        ", num_open)
    println("Total chargers:       ", total_chargers)
    println("Total fixed cost:     \$", round(total_fixed;      digits=2))
    println("Total charger cost:   \$", round(total_var;        digits=2))
    println("Detour+env cost:      \$", round(total_detour_cost; digits=2))
    println("Unmet demand penalty: \$", round(total_unmet_penalty; digits=2))
    println("Objective value:      \$", round(obj_val;          digits=2))
    println("Total demand (exp.):  ", round(total_demand;       digits=2))
    println("Served demand:        ", round(total_served;       digits=2))
    println("Unmet demand:         ", round(total_unmet;        digits=2))
    println("Served fraction:      ", round(served_fraction;    digits=3))
    println("Avg detour distance:  ",
            round(avg_detour; digits=3),
            " km per served demand unit")
    println("----------------------------------------------------")

    # =====================================================
    # Build a DataFrame with ALL decisions
    # =====================================================
    decisions = DataFrame(
        model_type = String[],
        R_max      = Float64[],
        var_type   = String[],
        ZIP        = Union{Missing,Int}[],
        station_id = Union{Missing,Int}[],
        value      = Float64[],
    )

    # y_j decisions (station open)
    for j in J
        push!(decisions, (
            "baseline",         # model_type
            R_max_km,           # R_max
            "y",                # var_type
            missing,            # ZIP not applicable
            station_ids[j],     # station_id
            value(y[j])         # value
        ))
    end

    # z_j decisions (chargers at station j)
    for j in J
        push!(decisions, (
            "baseline",
            R_max_km,
            "z",
            missing,
            station_ids[j],
            value(z[j])
        ))
    end

    # u_i decisions (unmet fraction at ZIP i)
    # If you don't have u[i] in your model, you can comment this loop out.
    for (i, zcode) in enumerate(zip_codes)
        push!(decisions, (
            "baseline",
            R_max_km,
            "u",
            zcode,         # ZIP
            missing,       # station_id
            value(u[i])    # unmet fraction
        ))
    end

    # x_ij decisions (fraction of demand i served by station j)
    # To avoid a huge file, we drop very small values.
    tol = 1e-6
    for (i,j) in edges
        x_val = value(x[(i,j)])
        if x_val > tol
            push!(decisions, (
                "baseline",
                R_max_km,
                "x",
                zip_codes[i],    # ZIP
                station_ids[j],  # station_id
                x_val
            ))
        end
    end

    # (optional) keep your old chosen_stations / assignments exports too
    info_map = Dict(row.ID => row for row in eachrow(stations_small))

    chosen_stations = DataFrame(
        station_id   = Int[],
        status_code  = String[],
        total_evse   = Float64[],
        chargers_opt = Float64[]
    )

    for j in open_sites_idx
        sid = station_ids[j]
        row = info_map[sid]
        push!(chosen_stations, (
            sid,
            row."Status Code",
            row.total_evse,
            value(z[j])
        ))
    end

    assignments = DataFrame(
        ZIP         = Int[],
        station_id  = Int[],
        frac_demand = Float64[]
    )
    for (i,j) in edges
        x_val = value(x[(i,j)])
        if x_val > 1e-4
            push!(assignments, (zip_codes[i], station_ids[j], x_val))
        end
    end

    if write_outputs
        suffix = isempty(tag) ? "R$(Int(R_max_km))" : tag
        CSV.write("chosen_stations_$(suffix).csv", chosen_stations)
        CSV.write("assignments_$(suffix).csv", assignments)
    end

    return (
        model = model,
        decisions = decisions,
        chosen_stations = chosen_stations,
        assignments = assignments,
        summary = (
            R_max = R_max_km,
            open_sites = num_open,
            total_chargers = total_chargers,
            total_fixed = total_fixed,
            total_var = total_var,
            total_detour_cost = total_detour_cost,
            total_unmet_penalty = total_unmet_penalty,
            total_demand = total_demand,
            served_demand = total_served,
            unmet_demand = total_unmet,
            served_fraction = served_fraction,
            avg_detour_km = avg_detour,
            objective = obj_val,
            term_status = term_status
        )
    )
end

solve_baseline_model (generic function with 1 method)

In [4]:
Rmax_values = [15.0]
results = Dict{Float64,Any}()

for R in Rmax_values
    println("\n=== Solving baseline model for R_max = $R km ===")
    res = solve_baseline_model(R; write_outputs = true)
    results[R] = res
end


=== Solving baseline model for R_max = 15.0 km ===
R_max = 15.0 km → feasible (i,j) pairs: 186729
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Termination status: OPTIMAL
Objective value:    3.102339979782643e9
---- Baseline summary for R_max = 15.0 km ----
Open stations:        4660
Total chargers:       45815.0
Total fixed cost:     $1.165e8
Total charger cost:   $6.87225e8
Detour+env cost:      $353979.78
Unmet demand penalty: $2.298261e9
Objective value:      $3.10233997978e9
Total demand (exp.):  320332.0
Served demand:        167114.6
Unmet demand:         153217.4
Served fraction:      0.522
Avg detour distance:  2.971 km per served demand unit
----------------------------------------------------


# Adaptive (2025)

In [4]:
using CSV
using DataFrames
using JuMP
using Gurobi
using Statistics
using Dates   # for timestamp
using FileIO   # for mkpath

function solve_adaptive_model(R_max_km::Float64;
                              write_outputs::Bool = false,
                              tag::String = "")

    λ_unmet = 15000.0   # SAME AS BASELINE

    # ---------- Feasible (i,j) edges within R_max ----------
    edges = Tuple{Int,Int}[]
    for i in I, j in J
        if d[i,j] <= R_max_km
            push!(edges, (i,j))
        end
    end

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    println("R_max = $(R_max_km) km → feasible (i,j) pairs: ", length(edges))

    # ---------- Build JuMP model ----------
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    set_optimizer_attribute(model, "TimeLimit", 600.0)
    set_optimizer_attribute(model, "MIPGap", 0.01)

    # First-stage decisions
    @variable(model, y[j in J], Bin)
    @variable(model, z[j in J] >= 0, Int)

    # Second-stage scenario-dependent
    @variable(model, x[(i,j) in edges, s in 1:S] >= 0)
    @variable(model, 0 .<= u[i in I, s in 1:S] .<= 1)

    # ---------- Constraints ----------
    # Demand balance per scenario
    @constraint(model, demand_balance[i in I, s in 1:S],
        sum(x[(i,j), s] for j in N_i[i]) + u[i,s] == 1
    )

    # Capacity: same as baseline but per scenario
    @constraint(model, capacity[j in J, s in 1:S],
        sum(demand[i,s] * x[(i,j), s] for i in N_j[j]) <= μ[j] * z[j]
    )

    # Assignment link
    @constraint(model, assignment_link[(i,j) in edges, s in 1:S],
        x[(i,j), s] <= y[j]
    )

    # Charger/opening links
    @constraint(model, charger_link[j in J], z[j] <= M * y[j])
    @constraint(model, charger_cap[j in J], z[j] <= max_chargers_per_station)

    # Budget (same)
    @constraint(model, budget,
        sum(F[j] * y[j] + v[j] * z[j] for j in J) <= B
    )

    # ---------- Objective: expected cost ----------
    @objective(model, Min,
        sum(F[j] * y[j] + v[j] * z[j] for j in J)
        +
        sum(prob[s] * cost_per_km * d[i,j] * demand[i,s] * x[(i,j),s]
            for (i,j) in edges, s in 1:S)
        +
        sum(prob[s] * λ_unmet * demand[i,s] * u[i,s]
            for i in I, s in 1:S)
    )

    optimize!(model)

    term_status = termination_status(model)
    obj_val = objective_value(model)

    println("Termination status: ", term_status)
    println("Objective value:    ", obj_val)

    # =====================================================
    # EXTRACT RESULTS (same as baseline)
    # =====================================================

    open_sites_idx = [j for j in J if value(y[j]) > 0.5]
    total_chargers = sum(value(z[j]) for j in J)

    # Expected costs
    total_fixed = sum(F[j] * value(y[j]) for j in J)
    total_var   = sum(v[j] * value(z[j]) for j in J)

    total_detour_cost =
        sum(prob[s] * cost_per_km * d[i,j] * demand[i,s] * value(x[(i,j),s])
            for (i,j) in edges, s in 1:S)

    total_unmet_penalty =
        sum(prob[s] * λ_unmet * demand[i,s] * value(u[i,s])
            for i in I, s in 1:S)

    # Total expected demand served
    total_served =
        sum(prob[s] * demand[i,s] * (1 - value(u[i,s])) for i in I, s in 1:S)

    total_unmet =
        sum(prob[s] * demand[i,s] * value(u[i,s]) for i in I, s in 1:S)

    served_fraction = total_served / (total_served + total_unmet)

    println("---- Adaptive summary for R_max = $(R_max_km) km ----")
    println("Open stations:        ", length(open_sites_idx))
    println("Total chargers:       ", total_chargers)
    println("Total fixed cost:     \$", round(total_fixed; digits=2))
    println("Total charger cost:   \$", round(total_var; digits=2))
    println("Detour+env cost:      \$", round(total_detour_cost; digits=2))
    println("Unmet demand penalty: \$", round(total_unmet_penalty; digits=2))
    println("Expected objective:   \$", round(obj_val; digits=2))
    println("Expected served:      ", round(total_served; digits=2))
    println("Expected unmet:       ", round(total_unmet; digits=2))
    println("Served fraction:      ", round(served_fraction; digits=3))
    println("------------------------------------------------------")

    # =====================================================
    # Build decisions DataFrame (same as baseline)
    # =====================================================
    decisions = DataFrame(
        model_type = String[],
        R_max      = Float64[],
        var_type   = String[],
        ZIP        = Union{Missing,Int}[],
        station_id = Union{Missing,Int}[],
        value      = Float64[],
        scenario   = Union{Missing,Int}[]
    )

    # y_j (binary)
    for j in J
        push!(decisions, ("adaptive", R_max_km, "y",
            missing, station_ids[j], value(y[j]), missing))
    end

    # z_j (chargers)
    for j in J
        push!(decisions, ("adaptive", R_max_km, "z",
            missing, station_ids[j], value(z[j]), missing))
    end

    # u_i,s (unmet)
    for i in I, s in 1:S
        push!(decisions, ("adaptive", R_max_km, "u",
            zip_codes[i], missing, value(u[i,s]), s))
    end

    # x_i,j,s (flows)
    tol = 1e-6
    for (i,j) in edges, s in 1:S
        xv = value(x[(i,j), s])
        if xv > tol
            push!(decisions, ("adaptive", R_max_km, "x",
                zip_codes[i], station_ids[j], xv, s))
        end
    end

    # =====================================================
    # chosen_stations DataFrame (same as baseline)
    # =====================================================
    info_map = Dict(row.ID => row for row in eachrow(stations_small))

    chosen_stations = DataFrame(
        station_id   = Int[],
        status_code  = String[],
        total_evse   = Float64[],
        chargers_opt = Float64[]
    )

    for j in open_sites_idx
        sid = station_ids[j]
        row = info_map[sid]
        push!(chosen_stations, (
            sid,
            row."Status Code",
            row.total_evse,
            value(z[j])
        ))
    end

    # =====================================================
    # assignments DataFrame
    # =====================================================
    assignments = DataFrame(
        ZIP         = Int[],
        station_id  = Int[],
        scenario    = Int[],
        frac_demand = Float64[]
    )

    for (i,j) in edges, s in 1:S
        xv = value(x[(i,j), s])
        if xv > 1e-4
            push!(assignments, (
                zip_codes[i],
                station_ids[j],
                s,
                xv
            ))
        end
    end

    # =====================================================
    # Write outputs to directory
    # =====================================================
    if write_outputs
        outdir = "../data/adaptive"
        mkpath(outdir)

        tag_clean = isempty(tag) ? "R$(Int(R_max_km))" : tag

        CSV.write("$outdir/chosen_stations_adaptive_$(tag_clean).csv", chosen_stations)
        CSV.write("$outdir/assignments_adaptive_$(tag_clean).csv", assignments)
        CSV.write("$outdir/decisions_adaptive_$(tag_clean).csv", decisions)

        summary_df = DataFrame(
            R_max = R_max_km,
            open_sites = length(open_sites_idx),
            total_chargers = total_chargers,
            total_fixed = total_fixed,
            total_var = total_var,
            total_detour_cost = total_detour_cost,
            total_unmet_penalty = total_unmet_penalty,
            expected_served = total_served,
            expected_unmet = total_unmet,
            served_fraction = served_fraction,
            objective = obj_val,
            term_status = string(term_status)
        )
        CSV.write("$outdir/summary_adaptive_$(tag_clean).csv", summary_df)
    end

    return (
        model = model,
        decisions = decisions,
        chosen_stations = chosen_stations,
        assignments = assignments
    )
end


solve_adaptive_model (generic function with 1 method)

In [5]:
println("\n=== START ADAPTIVE MODEL 1204 ===")

adaptive = CSV.read("../data/adaptive_1204.csv", DataFrame)
rename!(adaptive, Dict("ZIP Code" => "ZIP"))

# Ensure ZIP is Int
if eltype(adaptive.ZIP) <: AbstractFloat
    adaptive.ZIP = round.(Int, adaptive.ZIP)
elseif eltype(adaptive.ZIP) <: AbstractString
    adaptive.ZIP = parse.(Int, strip.(adaptive.ZIP))
end

# Keep only ZIPs used in the model
adaptive = adaptive[in.(adaptive.ZIP, Ref(Set(zip_codes))), :]

println("Adaptive rows after ZIP filter = ", nrow(adaptive))

scenario_ids = sort(unique(adaptive.scenario))
S = length(scenario_ids)
println("Scenarios found: ", scenario_ids)

scen_to_idx = Dict(sid => s for (s, sid) in enumerate(scenario_ids))
zip_to_i    = Dict(z => i for (i, z) in enumerate(zip_codes))

# demand[i,s] = 0.25 * predicted EV stock in 2025
sessions_per_EV_per_day = 0.25
demand = zeros(Float64, nI, S)
prob   = zeros(Float64, S)

for row in eachrow(adaptive)
    i = zip_to_i[row.ZIP]
    s = scen_to_idx[row.scenario]
    ev = row.demand_2025             # this is EV stock, same meaning as baseline N
    demand[i, s] = sessions_per_EV_per_day * ev
end

# Scenario weights
for sid in scenario_ids
    s = scen_to_idx[sid]
    w = adaptive.scenario_weight[adaptive.scenario .== sid]
    prob[s] = mean(w)
end
prob ./= sum(prob)

println("Scenario probabilities: ", prob)



=== START ADAPTIVE MODEL 1204 ===
Adaptive rows after ZIP filter = 3177
Scenarios found: [1, 2, 3]
Scenario probabilities: [0.30605799999999983, 0.3110439999999998, 0.3828980000000003]


In [6]:
function solve_adaptive_model(R_max_km::Float64;
    write_outputs::Bool = false,
    tag::String = "")

λ_unmet = 15000.0   # same as baseline

# ---------- Feasible (i,j) edges within R_max ----------
edges = Tuple{Int,Int}[]
for i in I, j in J
if d[i,j] <= R_max_km
push!(edges, (i,j))
end
end

N_i = Dict(i => Int[] for i in I)
N_j = Dict(j => Int[] for j in J)
for (i,j) in edges
push!(N_i[i], j)
push!(N_j[j], i)
end

println("R_max = $(R_max_km) km → feasible (i,j) pairs: ", length(edges))

# ---------- Build JuMP model ----------
model = Model(Gurobi.Optimizer)
set_silent(model)
set_optimizer_attribute(model, "TimeLimit", 600.0)
set_optimizer_attribute(model, "MIPGap", 0.01)

# First-stage decisions (scenario-independent)
@variable(model, y[j in J], Bin)         # open station j
@variable(model, z[j in J] >= 0, Int)    # chargers at j

# Second-stage (scenario-dependent)
@variable(model, x[edges, s in 1:S] >= 0)              # fraction of demand i served by j in scenario s
@variable(model, 0 .<= u[i in I, s in 1:S] .<= 1)      # unmet fraction at i in scenario s

# ---------- Constraints ----------
# Demand balance: served + unmet = 1 for each i,s
@constraint(model, demand_balance[i in I, s in 1:S],
sum(x[(i,j), s] for j in N_i[i]) + u[i,s] == 1
)

# Capacity: for each j,s
@constraint(model, capacity[j in J, s in 1:S],
sum(demand[i,s] * x[(i,j), s] for i in N_j[j]) <= μ[j] * z[j]
)

# Assignment link: x <= y
@constraint(model, assignment_link[(i,j) in edges, s in 1:S],
x[(i,j), s] <= y[j]
)

# Charger–opening link + hard cap
@constraint(model, charger_link[j in J],
z[j] <= M * y[j]
)
@constraint(model, charger_cap[j in J],
z[j] <= max_chargers_per_station
)

# Budget (same as baseline)
@constraint(model, budget,
sum(F[j] * y[j] + v[j] * z[j] for j in J) <= B
)

# ---------- Objective: expected cost ----------
@objective(model, Min,
# infrastructure cost (scenario-independent)
sum(F[j] * y[j] + v[j] * z[j] for j in J)
+
# expected detour+environment cost
sum(prob[s] * cost_per_km * d[i,j] * demand[i,s] * x[(i,j), s]
for (i,j) in edges, s in 1:S)
+
# expected unmet demand penalty
sum(prob[s] * λ_unmet * demand[i,s] * u[i,s] for i in I, s in 1:S)
)

# ---------- Solve ----------
optimize!(model)

term_status = termination_status(model)
println("Termination status: ", term_status)
obj_val = objective_value(model)
println("Objective value:    ", obj_val)

# =====================================================
#  Extract summary stats (same style as baseline)
# =====================================================

# Stations opened and chargers
open_sites_idx = [j for j in J if value(y[j]) > 0.5]
num_open = length(open_sites_idx)
total_chargers = sum(value(z[j]) for j in J)

# Costs
total_fixed = sum(F[j] * value(y[j]) for j in J)
total_var   = sum(v[j] * value(z[j]) for j in J)

total_detour_cost =
sum(prob[s] * cost_per_km * d[i,j] * demand[i,s] * value(x[(i,j), s])
for (i,j) in edges, s in 1:S)

total_unmet_penalty =
sum(prob[s] * λ_unmet * demand[i,s] * value(u[i,s])
for i in I, s in 1:S)

# Expected served / unmet demand
total_served =
sum(prob[s] * demand[i,s] * (1.0 - value(u[i,s])) for i in I, s in 1:S)

total_unmet =
sum(prob[s] * demand[i,s] * value(u[i,s]) for i in I, s in 1:S)

total_demand = total_served + total_unmet
served_fraction = total_served / max(total_demand, 1e-9)

# Average detour in km: detour cost / (cost_per_km * served demand)
avg_detour =
total_detour_cost / (cost_per_km * max(total_served, 1e-9))

println("---- Adaptive summary for R_max = $(R_max_km) km ----")
println("Open stations:        ", num_open)
println("Total chargers:       ", total_chargers)
println("Total fixed cost:     \$", round(total_fixed;       digits=2))
println("Total charger cost:   \$", round(total_var;         digits=2))
println("Detour+env cost:      \$", round(total_detour_cost; digits=2))
println("Unmet demand penalty: \$", round(total_unmet_penalty; digits=2))
println("Expected objective:   \$", round(obj_val;           digits=2))
println("Expected served:      ",  round(total_served;       digits=2))
println("Expected unmet:       ",  round(total_unmet;        digits=2))
println("Served fraction:      ",  round(served_fraction;    digits=3))
println("Avg detour (km):      ",  round(avg_detour;         digits=3))
println("------------------------------------------------------")

# =====================================================
#  Build decisions DataFrame
# =====================================================
decisions = DataFrame(
model_type = String[],
R_max      = Float64[],
var_type   = String[],
ZIP        = Union{Missing,Int}[],
station_id = Union{Missing,Int}[],
value      = Float64[],
scenario   = Union{Missing,Int}[]
)

# y_j
for j in J
push!(decisions, (
"adaptive", R_max_km, "y",
missing, station_ids[j], value(y[j]), missing
))
end

# z_j
for j in J
push!(decisions, (
"adaptive", R_max_km, "z",
missing, station_ids[j], value(z[j]), missing
))
end

# u_i,s
for i in I, s in 1:S
push!(decisions, (
"adaptive", R_max_km, "u",
zip_codes[i], missing, value(u[i,s]), s
))
end

# x_i,j,s  (only store non-tiny)
tol = 1e-6
for (i,j) in edges, s in 1:S
xv = value(x[(i,j), s])
if xv > tol
push!(decisions, (
"adaptive", R_max_km, "x",
zip_codes[i], station_ids[j], xv, s
))
end
end

# =====================================================
# chosen_stations & assignments
# =====================================================
info_map = Dict(row.ID => row for row in eachrow(stations_small))

chosen_stations = DataFrame(
station_id   = Int[],
status_code  = String[],
total_evse   = Float64[],
chargers_opt = Float64[]
)

for j in open_sites_idx
sid = station_ids[j]
row = info_map[sid]
push!(chosen_stations, (
sid,
row."Status Code",
row.total_evse,
value(z[j])
))
end

assignments = DataFrame(
ZIP         = Int[],
station_id  = Int[],
scenario    = Int[],
frac_demand = Float64[]
)

for (i,j) in edges, s in 1:S
xv = value(x[(i,j), s])
if xv > 1e-4
push!(assignments, (
zip_codes[i],
station_ids[j],
s,
xv
))
end
end

# =====================================================
# Optional: write outputs
# =====================================================
if write_outputs
outdir = "../data/adaptive"
mkpath(outdir)

suffix = isempty(tag) ? "R$(Int(R_max_km))" : tag

CSV.write("$outdir/chosen_stations_adaptive_$(suffix).csv", chosen_stations)
CSV.write("$outdir/assignments_adaptive_$(suffix).csv", assignments)
CSV.write("$outdir/decisions_adaptive_$(suffix).csv", decisions)

summary_df = DataFrame(
R_max = R_max_km,
open_sites = num_open,
total_chargers = total_chargers,
total_fixed = total_fixed,
total_var = total_var,
total_detour_cost = total_detour_cost,
total_unmet_penalty = total_unmet_penalty,
expected_served = total_served,
expected_unmet = total_unmet,
served_fraction = served_fraction,
avg_detour_km = avg_detour,
objective = obj_val,
term_status = string(term_status)
)
CSV.write("$outdir/summary_adaptive_$(suffix).csv", summary_df)
end

# Canonical summary object for aggregation
summary = (
R_max = R_max_km,
open_sites = num_open,
total_chargers = total_chargers,
total_fixed = total_fixed,
total_var = total_var,
total_detour_cost = total_detour_cost,
total_unmet_penalty = total_unmet_penalty,
total_demand = total_demand,
served_demand = total_served,
unmet_demand = total_unmet,
served_fraction = served_fraction,
avg_detour_km = avg_detour,
objective = obj_val,
term_status = string(term_status)
)

return (
model = model,
decisions = decisions,
chosen_stations = chosen_stations,
assignments = assignments,
summary = summary
)
end

solve_adaptive_model (generic function with 1 method)

In [8]:
Rmax_values = [15.0]
adaptive_results = Dict{Float64,Any}()

for R in Rmax_values
    println("\n=== Solving ADAPTIVE model for R_max = $R km ===")
    adaptive_results[R] = solve_adaptive_model(R; write_outputs=false)
end



=== Solving ADAPTIVE model for R_max = 15.0 km ===
R_max = 15.0 km → feasible (i,j) pairs: 186729
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Termination status: OPTIMAL
Objective value:    5.091850143331971e9
---- Adaptive summary for R_max = 15.0 km ----
Open stations:        5315
Total chargers:       52918.0
Total fixed cost:     $1.32875e8
Total charger cost:   $7.9377e8
Detour+env cost:      $369599.6
Unmet demand penalty: $4.16483554373e9
Expected objective:   $5.09185014333e9
Expected served:      186738.52
Expected unmet:       277655.7
Served fraction:      0.402
Avg detour (km):      2.776
------------------------------------------------------


# Deterministic

In [7]:
function solve_deterministic_point(R_max_km::Float64)

    s0 = 2   # Scenario 2 is the point-forecast

    println("\n=== Solving DETERMINISTIC (Scenario 2) for R_max = $R_max_km ===")

    # ---- Feasible edges ----
    edges = [(i,j) for i in I, j in J if d[i,j] <= R_max_km]

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    λ_unmet = 15000.0

    model = Model(Gurobi.Optimizer)
    set_silent(model)
    set_optimizer_attribute(model, "TimeLimit", 600.0)
    set_optimizer_attribute(model, "MIPGap", 0.01)

    # First-stage decisions
    @variable(model, y[j in J], Bin)
    @variable(model, z[j in J] >= 0, Int)

    # Second-stage (single scenario)
    @variable(model, x[(i,j) in edges] >= 0)
    @variable(model, 0 .<= u[i in I] .<= 1)

    # Constraints
    @constraint(model, [i in I],
        sum(x[(i,j)] for j in N_i[i]) + u[i] == 1
    )

    @constraint(model, [j in J],
        sum(demand[i,s0] * x[(i,j)] for i in N_j[j]) <= μ[j] * z[j]
    )

    @constraint(model, [i in I, j in N_i[i]],
        x[(i,j)] <= y[j]
    )

    @constraint(model, [j in J], z[j] <= M * y[j])
    @constraint(model, [j in J], z[j] <= max_chargers_per_station)

    @constraint(model,
        sum(F[j]*y[j] + v[j]*z[j] for j in J) <= B
    )

    # Objective
    @objective(model, Min,
        sum(F[j]*y[j] + v[j]*z[j] for j in J)
        +
        sum(cost_per_km * d[i,j] * demand[i,s0] * x[(i,j)] for (i,j) in edges)
        +
        sum(λ_unmet * demand[i,s0] * u[i] for i in I)
    )

    optimize!(model)

    term_status = termination_status(model)
    obj_val = objective_value(model)

    # Extract first-stage decisions
    y_opt = Dict(j => value(y[j]) for j in J)
    z_opt = Dict(j => value(z[j]) for j in J)

    println("Deterministic objective = ", obj_val)

    return (model = model, y_opt = y_opt, z_opt = z_opt)
end


solve_deterministic_point (generic function with 1 method)

In [8]:
function evaluate_deterministic_first_stage(y_opt, z_opt, R_max_km::Float64)

    println("\n=== Evaluating deterministic decisions under all scenarios ===")

    S_costs = zeros(S)
    served  = zeros(S)
    unmet   = zeros(S)

    # Build edges again
    edges = [(i,j) for i in I, j in J if d[i,j] <= R_max_km]

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    λ_unmet = 15000.0

    # Evaluate deterministic decisions under each scenario
    for s in 1:S

        model = Model(Gurobi.Optimizer)
        set_silent(model)

        @variable(model, x[(i,j) in edges] >= 0)
        @variable(model, 0 .<= u[i in I] .<= 1)

        @constraint(model, demand_balance[i in I],
            sum(x[(i,j)] for j in N_i[i]) + u[i] == 1
        )

        @constraint(model, capacity[j in J],
            sum(demand[i,s] * x[(i,j)] for i in N_j[j]) <= μ[j] * z_opt[j]
        )

        @constraint(model, assignment_link[i in I, j in N_i[i]],
            x[(i,j)] <= y_opt[j]
        )

        @objective(model, Min,
            sum(F[j] * y_opt[j] + v[j] * z_opt[j] for j in J)
            +
            sum(cost_per_km * d[i,j] * demand[i,s] * x[(i,j)] for (i,j) in edges)
            +
            sum(λ_unmet * demand[i,s] * u[i] for i in I)
        )

        optimize!(model)

        S_costs[s] = objective_value(model)
        served[s]  = sum(demand[i,s] * (1 - value(u[i])) for i in I)
        unmet[s]   = sum(demand[i,s] * value(u[i])       for i in I)

        println(" Scenario $s cost = ", S_costs[s])
    end

    # Expected cost across scenarios
    expected_cost = sum(prob[s] * S_costs[s] for s in 1:S)
    println("Expected deterministic cost = ", expected_cost)

    # --------------------------------------------
    # Build consistent summary object
    # --------------------------------------------

    # Infrastructure (same across scenarios)
    total_fixed = sum(F[j] * y_opt[j] for j in J)
    total_var   = sum(v[j] * z_opt[j] for j in J)

    # Expected unmet penalty
    total_unmet_penalty =
        sum(prob[s] * unmet[s] * λ_unmet for s in 1:S)

    # Expected served / unmet demand
    total_served = sum(prob[s] * served[s] for s in 1:S)
    total_unmet  = sum(prob[s] * unmet[s]  for s in 1:S)
    total_demand = total_served + total_unmet

    served_fraction = total_served / (total_demand + 1e-9)

    # Expected detour cost = total expected cost minus fixed, var, and unmet penalty
    total_detour_cost =
        expected_cost - total_fixed - total_var - total_unmet_penalty

    # Average detour in km: detour cost / (cost_per_km * served demand)
    avg_detour =
        total_detour_cost / (cost_per_km * max(total_served, 1e-9))

    # Number of open sites and total chargers from y_opt, z_opt
    open_sites     = sum(y_opt[j] > 0.5 for j in J)
    total_chargers = sum(z_opt[j]       for j in J)

    summary = (
        R_max = R_max_km,
        open_sites = open_sites,
        total_chargers = total_chargers,
        total_fixed = total_fixed,
        total_var = total_var,
        total_detour_cost = total_detour_cost,
        total_unmet_penalty = total_unmet_penalty,
        total_demand = total_demand,
        served_demand = total_served,
        unmet_demand = total_unmet,
        served_fraction = served_fraction,
        avg_detour_km = avg_detour,
        objective = expected_cost,
        term_status = "EVALUATED"
    )

    return (
        S_costs = S_costs,
        served_per_scenario = served,
        unmet_per_scenario = unmet,
        expected_cost = expected_cost,
        summary = summary
    )
end

evaluate_deterministic_first_stage (generic function with 1 method)

In [11]:
det = solve_deterministic_point(15.0)
det_eval = evaluate_deterministic_first_stage(det.y_opt, det.z_opt, 15.0)


=== Solving DETERMINISTIC (Scenario 2) for R_max = 15.0 ===
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Deterministic objective = 5.165779879186521e9

=== Evaluating deterministic decisions under all scenarios ===
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
 Scenario 1 cost = 3.7395668695516396e9
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
 Scenario 2 cost = 5.165779879186643e9
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
 Scenario 3 cost = 6.124700414048168e9
Expected deterministic cost = 5.096444732841182e9


(S_costs = [3.7395668695516396e9, 5.165779879186643e9, 6.124700414048168e9], served_per_scenario = [177280.85915467681, 188204.84637529092, 188277.95000000022], unmet_per_scenario = [189045.43432411223, 284126.2029912688, 348056.5046501814], expected_cost = 5.096444732841182e9, summary = (R_max = 15.0, open_sites = 5191, total_chargers = 51583.0, total_fixed = 1.29775e8, total_var = 7.73745e8, total_detour_cost = 353366.75422382355, total_unmet_penalty = 4.192571366086958e9, total_demand = 464394.2216653486, served_demand = 184889.46392621813, unmet_demand = 279504.7577391305, served_fraction = 0.39813041442073027, avg_detour_km = 2.680550504996552, objective = 5.096444732841182e9, term_status = "EVALUATED"))

# Wait and See

In [9]:
function solve_wait_and_see(R_max_km::Float64)

    println("\n=== Solving WAIT-AND-SEE for R_max = $R_max_km ===")

    # ------------------------------------------
    # Build feasible edges
    # ------------------------------------------
    edges = [(i,j) for i in I, j in J if d[i,j] <= R_max_km]

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    λ_unmet = 15000.0

    # ------------------------------------------
    # Storage across scenarios
    # ------------------------------------------
    cost_scenario = zeros(S)
    served_s      = zeros(S)
    unmet_s       = zeros(S)

    # y_WS[s] and z_WS[s]: scenario-specific first-stage decisions
    y_WS = Dict{Int, Vector{Float64}}()
    z_WS = Dict{Int, Vector{Float64}}()

    assignments = DataFrame(
        ZIP         = Int[],
        station_id  = Int[],
        scenario    = Int[],
        frac_demand = Float64[]
    )

    # ------------------------------------------
    # Solve S independent models (one per scenario)
    # ------------------------------------------
    for s in 1:S
        println("  -- Scenario $s --")

        model = Model(Gurobi.Optimizer)
        set_silent(model)
        set_optimizer_attribute(model, "TimeLimit", 600.0)
        set_optimizer_attribute(model, "MIPGap", 0.01)

        @variable(model, y[j in J], Bin)
        @variable(model, z[j in J] >= 0, Int)
        @variable(model, x[(i,j) in edges] >= 0)
        @variable(model, 0 .<= u[i in I] .<= 1)

        @constraint(model, demand_balance[i in I],
            sum(x[(i,j)] for j in N_i[i]) + u[i] == 1
        )

        @constraint(model, capacity[j in J],
            sum(demand[i,s] * x[(i,j)] for i in N_j[j]) <= μ[j] * z[j]
        )

        @constraint(model, assignment_link[i in I, j in N_i[i]],
            x[(i,j)] <= y[j]
        )

        @constraint(model, charger_link[j in J], z[j] <= M * y[j])
        @constraint(model, charger_cap[j in J], z[j] <= max_chargers_per_station)

        @constraint(model, budget,
            sum(F[j] * y[j] + v[j] * z[j] for j in J) <= B
        )

        @objective(model, Min,
            sum(F[j]*y[j] + v[j]*z[j] for j in J) +
            sum(cost_per_km * d[i,j] * demand[i,s] * x[(i,j)] for (i,j) in edges) +
            sum(λ_unmet * demand[i,s] * u[i] for i in I)
        )

        optimize!(model)

        # --- store results for scenario s
        cost_scenario[s] = objective_value(model)
        served_s[s] = sum(demand[i,s] * (1 - value(u[i])) for i in I)
        unmet_s[s]  = sum(demand[i,s] * value(u[i])       for i in I)

        y_WS[s] = [value(y[j]) for j in J]
        z_WS[s] = [value(z[j]) for j in J]

        # assignments for scenario s
        for (i,j) in edges
            xv = value(x[(i,j)])
            if xv > 1e-4
                push!(assignments, (
                    zip_codes[i],
                    station_ids[j],
                    s,
                    xv
                ))
            end
        end

        println("     WS scenario cost = ", cost_scenario[s])
    end

    # ------------------------------------------
    # Expected cost and demand across scenarios
    # ------------------------------------------
    expected_WS = sum(prob[s] * cost_scenario[s] for s in 1:S)
    println("\nExpected Wait-and-See cost = ", expected_WS)

    # ------------------------------------------
    # Build station list (scenario-wise)
    # ------------------------------------------
    chosen_stations = DataFrame(
        scenario     = Int[],
        station_id   = Int[],
        chargers_opt = Float64[]
    )

    for s in 1:S
        for j in J
            if y_WS[s][j] > 0.5
                push!(chosen_stations, (
                    s,
                    station_ids[j],
                    z_WS[s][j]
                ))
            end
        end
    end

    # ------------------------------------------
    # Compute expected fixed, variable, detour, unmet
    # ------------------------------------------
    total_fixed =
        sum(prob[s] * sum(F[j] * y_WS[s][j] for j in J) for s in 1:S)

    total_var =
        sum(prob[s] * sum(v[j] * z_WS[s][j] for j in J) for s in 1:S)

    total_unmet_penalty =
        sum(prob[s] * unmet_s[s] * λ_unmet for s in 1:S)

    served_exp = sum(prob[s] * served_s[s] for s in 1:S)
    unmet_exp  = sum(prob[s] * unmet_s[s]  for s in 1:S)

    total_demand = served_exp + unmet_exp
    served_fraction = served_exp / (total_demand + 1e-9)

    # Expected detour cost = total expected cost minus fixed, var, and unmet penalty
    total_detour_cost =
        expected_WS - total_fixed - total_var - total_unmet_penalty

    # Average detour in km
    avg_detour =
        total_detour_cost / (cost_per_km * max(served_exp, 1e-9))

    # Expected (probability-weighted) open sites and chargers
    open_sites_exp =
        sum(prob[s] * sum(y_WS[s][j] > 0.5 for j in J) for s in 1:S)

    total_chargers_exp =
        sum(prob[s] * sum(z_WS[s][j]       for j in J) for s in 1:S)

    # ------------------------------------------
    # Summary (canonical shape)
    # ------------------------------------------
    summary = (
        R_max = R_max_km,
        open_sites = open_sites_exp,
        total_chargers = total_chargers_exp,
        total_fixed = total_fixed,
        total_var = total_var,
        total_detour_cost = total_detour_cost,
        total_unmet_penalty = total_unmet_penalty,
        total_demand = total_demand,
        served_demand = served_exp,
        unmet_demand = unmet_exp,
        served_fraction = served_fraction,
        avg_detour_km = avg_detour,
        objective = expected_WS,
        term_status = "WAIT_AND_SEE"
    )

    # ------------------------------------------
    # Decisions DF (scenario-specific y and z)
    # ------------------------------------------
    decisions = DataFrame(
        model_type = String[],
        R_max      = Float64[],
        var_type   = String[],
        ZIP        = Union{Missing,Int}[],
        station_id = Union{Missing,Int}[],
        value      = Float64[],
        scenario   = Union{Missing,Int}[]
    )

    for s in 1:S, j in J
        push!(decisions, (
            "wait_and_see", R_max_km, "y",
            missing, station_ids[j], y_WS[s][j], s
        ))
        push!(decisions, (
            "wait_and_see", R_max_km, "z",
            missing, station_ids[j], z_WS[s][j], s
        ))
    end

    return (
        decisions = decisions,
        chosen_stations = chosen_stations,
        assignments = assignments,
        summary = summary
    )
end

solve_wait_and_see (generic function with 1 method)

In [13]:
ws = solve_wait_and_see(15.0)
ws.summary


=== Solving WAIT-AND-SEE for R_max = 15.0 ===
  -- Scenario 1 --
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
     WS scenario cost = 3.688157801012732e9
  -- Scenario 2 --
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
     WS scenario cost = 5.165779879186521e9
  -- Scenario 3 --
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
     WS scenario cost = 6.07521662490642e9

Expected Wait-and-See cost = 5.061763332247465e9


(R_max = 15.0, open_sites = 5156.916848, total_chargers = 51187.018125999995, total_fixed = 1.2892292119999999e8, total_var = 7.6780527189e8, total_detour_cost = 369980.8929028511, total_unmet_penalty = 4.1646651582645626e9, total_demand = 464394.2216653486, served_demand = 186749.87778104446, unmet_demand = 277644.34388430417, served_fraction = 0.40213652338598566, avg_detour_km = 2.778621808695901, objective = 5.061763332247465e9, term_status = "WAIT_AND_SEE")

# Summary Table

In [10]:
using DataFrames

function build_ev_summary(base, det, ada, ws)

    return DataFrame(
        Model = [
            "Baseline 2022 (existing network)",
            "Deterministic 2025 (point forecast)",
            "Adaptive 2025 (two-stage stochastic)",
            "Wait-and-See 2025"
        ],

        R_max = [
            base.R_max,
            det.R_max,
            ada.R_max,
            ws.R_max
        ],

        Objective_Cost = [
            base.objective,
            det.objective,
            ada.objective,
            ws.objective
        ],

        Total_Fixed_Cost = [
            base.total_fixed,
            det.total_fixed,
            ada.total_fixed,
            ws.total_fixed
        ],

        Total_Variable_Cost = [
            base.total_var,
            det.total_var,
            ada.total_var,
            ws.total_var
        ],

        Total_Detour_Cost = [
            base.total_detour_cost,
            det.total_detour_cost,
            ada.total_detour_cost,
            ws.total_detour_cost
        ],

        Total_Unmet_Penalty = [
            base.total_unmet_penalty,
            det.total_unmet_penalty,
            ada.total_unmet_penalty,
            ws.total_unmet_penalty
        ],

        Total_Demand = [
            base.total_demand,
            det.total_demand,
            ada.total_demand,
            ws.total_demand
        ],

        Expected_Served = [
            base.served_demand,
            det.served_demand,
            ada.served_demand,
            ws.served_demand
        ],

        Expected_Unmet = [
            base.unmet_demand,
            det.unmet_demand,
            ada.unmet_demand,
            ws.unmet_demand
        ],

        Served_Fraction = [
            base.served_fraction,
            det.served_fraction,
            ada.served_fraction,
            ws.served_fraction
        ],

        Avg_Detour_km = [
            base.avg_detour_km,
            det.avg_detour_km,
            ada.avg_detour_km,
            ws.avg_detour_km
        ],

        Open_Stations = [
            base.open_sites,
            det.open_sites,
            ada.open_sites,
            ws.open_sites
        ],

        Total_Chargers = [
            base.total_chargers,
            det.total_chargers,
            ada.total_chargers,
            ws.total_chargers
        ],

        Termination_Status = [
            string(base.term_status),
            string(det.term_status),
            string(ada.term_status),
            string(ws.term_status)
        ]
    )
end

build_ev_summary (generic function with 1 method)

In [11]:
Rmax = 15.0

# 1. Baseline 2022
base_res = solve_baseline_model(Rmax; write_outputs = false)
base_sum = base_res.summary

# 2. Deterministic 2025 (point forecast)
det_res  = solve_deterministic_point(Rmax)
det_eval = evaluate_deterministic_first_stage(det_res.y_opt, det_res.z_opt, Rmax)
det_sum  = det_eval.summary

# 3. Adaptive 2025
ada_res = solve_adaptive_model(Rmax; write_outputs = false, tag = "R15")
ada_sum = ada_res.summary

# 4. Wait-and-See 2025
ws_res = solve_wait_and_see(Rmax)
ws_sum = ws_res.summary

# 5. Build final comparison table
ev_table = build_ev_summary(base_sum, det_sum, ada_sum, ws_sum)

println(ev_table)

# Optional: save to CSV
CSV.write("../data/summary_all_models_R15.csv", ev_table)

R_max = 15.0 km → feasible (i,j) pairs: 186729
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Termination status: OPTIMAL
Objective value:    3.102339979782643e9
---- Baseline summary for R_max = 15.0 km ----
Open stations:        4660
Total chargers:       45815.0
Total fixed cost:     $1.165e8
Total charger cost:   $6.87225e8
Detour+env cost:      $353979.78
Unmet demand penalty: $2.298261e9
Objective value:      $3.10233997978e9
Total demand (exp.):  320332.0
Served demand:        167114.6
Unmet demand:         153217.4
Served fraction:      0.522
Avg detour distance:  2.971 km per served demand unit
----------------------------------------------------

=== Solving DETERMINISTIC (Scenario 2) for R_max = 15.0 ===
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Deterministic objective = 5.165779879186521e9

=== Evaluating deterministic decisions under all scenarios ===
Set parameter Username
Academi

"../data/summary_all_models_R15.csv"

In [35]:
using CSV, DataFrames, JuMP, Gurobi

# --------------------------------------------------------
# Assumed existing objects from your model code
# --------------------------------------------------------
# I           :: Vector / 1:nI  (indices of ZIPs)
# J           :: Vector / 1:nJ  (indices of stations)
# S           :: Int            (number of demand scenarios)
# zip_codes   :: Vector{Int}    (ZIP code for each i in I)
# station_ids :: Vector{Int}    (ID for each j in J)
# demand[i,s] :: Float64        (sessions/day)
# prob[s]     :: Float64        (scenario probabilities, sum = 1)
# d[i,j]      :: Float64        (distance matrix)
# μ[j]        :: Float64        (sessions/day per charger)
# B, F[j], v[j], M, max_chargers_per_station defined as before
# Rmax        :: Float64        (the R_max you solved with)

# Results already computed:
# det_res  = solve_deterministic_point(Rmax)
# det_eval = evaluate_deterministic_first_stage(det_res.y_opt, det_res.z_opt, Rmax)
# ada_res  = solve_adaptive_model(Rmax; write_outputs = false, tag = "R15")
# ws_res   = solve_wait_and_see(Rmax)

# --------------------------------------------------------
# 1. Helper: compute fraction served per ZIP from u[i,s]
# --------------------------------------------------------
function compute_phi_zip_from_u(u_mat::Array{Float64,2})
    # u_mat[i,s] = unmet fraction at ZIP index i in scenario s
    phi = Dict{Int,Float64}()
    nI, S = size(u_mat)

    for (i, z) in enumerate(zip_codes)
        num_unmet  = 0.0
        den_demand = 0.0
        for s in 1:S
            num_unmet  += prob[s] * demand[i,s] * u_mat[i,s]
            den_demand += prob[s] * demand[i,s]
        end
        phi[z] = den_demand > 0 ? 1.0 - num_unmet / den_demand : 0.0
    end

    return phi
end

# --------------------------------------------------------
# 2. Adaptive: u[i,s] directly from JuMP model
# --------------------------------------------------------
model_ad = ada_res.model
u_ad_var = model_ad[:u]        # u[i,s] variable

u_ad_mat = zeros(Float64, length(I), S)
for (i_idx, _) in enumerate(I), s in 1:S
    u_ad_mat[i_idx, s] = value(u_ad_var[i_idx, s])
end

phi_ad = compute_phi_zip_from_u(u_ad_mat)

# --------------------------------------------------------
# 3. Deterministic: rebuild second stage for each scenario,
#    using fixed first-stage (det_res.y_opt, det_res.z_opt)
# --------------------------------------------------------
u_det_mat = zeros(Float64, length(I), S)
λ_unmet = 15000.0

for s in 1:S
    println("Solving deterministic evaluation for scenario $s ...")

    # Feasible edges within Rmax
    edges = [(i,j) for i in I, j in J if d[i,j] <= Rmax]

    N_i = Dict(i => Int[] for i in I)
    N_j = Dict(j => Int[] for j in J)
    for (i,j) in edges
        push!(N_i[i], j)
        push!(N_j[j], i)
    end

    model = Model(Gurobi.Optimizer)
    set_silent(model)

    @variable(model, x[(i,j) in edges] >= 0)
    @variable(model, 0 .<= u[i in I] .<= 1)

    @constraint(model, [i in I],
        sum(x[(i,j)] for j in N_i[i]) + u[i] == 1
    )

    @constraint(model, [j in J],
        sum(demand[i,s] * x[(i,j)] for i in N_j[j]) <= μ[j] * det_res.z_opt[j]
    )

    @constraint(model, [i in I, j in N_i[i]],
        x[(i,j)] <= det_res.y_opt[j]
    )

    @objective(model, Min,
        sum(F[j]*det_res.y_opt[j] + v[j]*det_res.z_opt[j] for j in J) +
        sum(cost_per_km * d[i,j] * demand[i,s] * x[(i,j)] for (i,j) in edges) +
        sum(λ_unmet * demand[i,s] * u[i] for i in I)
    )

    optimize!(model)

    for (i_idx, i) in enumerate(I)
        u_det_mat[i_idx, s] = value(u[i])
    end
end

phi_det = compute_phi_zip_from_u(u_det_mat)

# --------------------------------------------------------
# 4. Wait-and-See: u[i,s] from ws_res.decisions DataFrame
# --------------------------------------------------------
dec_ws = ws_res.decisions  # columns: model_type, var_type, ZIP, scenario, value, ...

u_ws_mat = zeros(Float64, length(I), S)

for (i_idx, z) in enumerate(zip_codes)
    for s in 1:S
        row_u = filter(row ->
            row.model_type == "wait_and_see" &&
            row.var_type   == "u" &&
            row.scenario   == s &&
            row.ZIP        == z,
            dec_ws
        )
        u_ws_mat[i_idx, s] = isempty(row_u) ? 0.0 : row_u.value[1]
    end
end

phi_ws = compute_phi_zip_from_u(u_ws_mat)

println("phi_det: min=", minimum(values(phi_det)), ", max=", maximum(values(phi_det)))
println("phi_ad : min=", minimum(values(phi_ad)),  ", max=", maximum(values(phi_ad)))
println("phi_ws : min=", minimum(values(phi_ws)),  ", max=", maximum(values(phi_ws)))

# --------------------------------------------------------
# 5. Build ZIP–centroid table and export CSV
# --------------------------------------------------------
using CSV, DataFrames

# 1. Read your registration file
reg_raw = CSV.read("../data/cleaned_reg_1128.csv", DataFrame)

# 2. ZIP–centroid table (unique rows)
zip_meta = unique(select(reg_raw, :ZIP, :lat, :lon))

# 3. Ensure ZIP type matches zip_codes (Int)
zip_meta.ZIP = parse.(Int, string.(zip_meta.ZIP))

# 4. Build export_df for ZIPs actually in the model
export_df = DataFrame(ZIP = zip_codes)
export_df = leftjoin(export_df, zip_meta, on = :ZIP)

# 5. Attach per-ZIP fractions served
export_df[:, :phi_det] = [get(phi_det, z, 0.0) for z in export_df.ZIP]
export_df[:, :phi_ad]  = [get(phi_ad,  z, 0.0) for z in export_df.ZIP]
export_df[:, :phi_ws]  = [get(phi_ws,  z, 0.0) for z in export_df.ZIP]

# (Optional) check a few rows
first(export_df, 10) |> println

# 6. Write CSV
CSV.write("ev_zip_metrics_R15.csv", export_df)
println("Wrote ev_zip_metrics_R15.csv")

Solving deterministic evaluation for scenario 1 ...
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Solving deterministic evaluation for scenario 2 ...
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
Solving deterministic evaluation for scenario 3 ...
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-20
phi_det: min=0.0, max=1.0
phi_ad : min=0.0, max=1.0
phi_ws : min=0.0, max=1.0
10×6 DataFrame
 Row │ ZIP    lat       lon       phi_det    phi_ad     phi_ws  
     │ Int64  Float64?  Float64?  Float64    Float64    Float64 
─────┼──────────────────────────────────────────────────────────
   1 │ 10001   40.7484  -73.9967  0.949955   0.949955       1.0
   2 │ 10002   40.7152  -73.9877  0.469557   0.469557       1.0
   3 │ 10003   40.7313  -73.9892  0.597588   0.597588       1.0
   4 │ 10004   40.7143  -74.006   0.0697986  0.0567766      1.0
   5 │ 10006   40.7085  -74